# How to add memory of structured entities

One strategy for managing long conversation histories is to restrict what information is stored "long-term". For example, instead of storing messages verbatim, an application can persist selected facts that it learns during the conversation.

This guide demonstrates how to persist structured information that is scoped to individual users. We demonstrate two methods:

1. Using LangGraph's built-in [persistence layer](https://langchain-ai.github.io/langgraph/how-tos/persistence/);
2. Using an external knowledge base.

## Using built-in persistence

LangGraph supports built-in persistence via a selection of backends, including in-memory, SQLite, and Postgres. See the persistence [documentation](https://langchain-ai.github.io/langgraph/how-tos/persistence/) for details on these. Below we will use a simple in-memory checkpointer.

Our strategy is to add fields to the state that are shared across threads (e.g., threads belonging to a specific user). We do this by compiling the graph with a [MemoryStore](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.store) and annotating a field of the state with `SharedValue`.

In the example below, we build a simple application that persists facts learned about the user.

In [1]:
from typing import Literal, TypedDict, Annotated
import uuid

from langgraph.graph.message import MessagesState
from langgraph.graph.state import StateGraph
from langgraph.store.memory import MemoryStore
from langgraph.managed.shared_value import SharedValue
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


class State(MessagesState):
    # We use an info key to track information
    # This is scoped to a user_id, so it will be information specific to each user
    info: Annotated[dict[str, dict], SharedValue.on("user_id")]


# We will give this as a tool to the agent
# This will let the agent call this tool to save a fact
class Info(TypedDict):
    """This tool should be called when you want to save a new fact about the user.

    Attributes:
        fact (str): A fact about the user.
        topic (str): The topic related the fact is about, i.e. Food, Location, Movies, etc.
    """

    fact: str
    topic: str


# This is the prompt we give the agent
# We will pass known info into the prompt
# We will tell it to use the Info tool to save more
prompt = """You are a helpful assistant that learns about users to provide better assistance.

Current user information:

{info}


Instructions:
1. Use the `Info` tool to save new information the user shares.
2. Save facts, opinions, preferences, and experiences.
3. Your goal: Improve assistance by building a user profile over time.

Remember: Every piece of information helps you serve the user better in future interactions.
"""


# We give the model access to the Info tool
model = ChatOpenAI().bind_tools([Info])


def call_model(state: State):
    """Call the model."""
    # The info value here is scoped to the user_id
    info = "\n".join([d["fact"] for d in state["info"].values()])
    # Format system prompt
    system_msg = prompt.format(info=info)
    # Call model
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": [response]}


# Routing function to decide what to do next
# If no tool calls, then we end
# If tool calls, then we update memory
def route(state) -> Literal["__end__", "update_memory"]:
    if len(state["messages"][-1].tool_calls) == 0:
        return "__end__"
    else:
        return "update_memory"


def update_memory(state: State):
    """Update the memory."""
    tool_calls = []
    memories = {}
    # Each tool call is a new memory to save
    for tc in state["messages"][-1].tool_calls:
        # We append ToolMessages (to pass back to the LLM)
        # This is needed because OpenAI requires each tool call be followed by a ToolMessage
        tool_calls.append(
            {"role": "tool", "content": "Saved!", "tool_call_id": tc["id"]}
        )
        # We create a new memory from this tool call
        memories[str(uuid.uuid4())] = {
            "fact": tc["args"]["fact"],
            "topic": tc["args"]["topic"],
        }
    # Return the messages and memories to update the state with
    return {"messages": tool_calls, "info": memories}


# This is the in memory checkpointer we will use
# We need this because we want to enable threads (conversations)
memory = MemorySaver()

# This is the in memory Key Value store
# This is needed to save the memories
kv = MemoryStore()

# Construct this relatively simple graph
graph = StateGraph(State)
graph.add_node(call_model)
graph.add_node(update_memory)
graph.add_edge("update_memory", "__end__")
graph.add_edge("__start__", "call_model")
graph.add_conditional_edges("call_model", route, ["__end__", "update_memory"])
graph = graph.compile(checkpointer=memory, store=kv)

At the start, the application functions like a simple chatbot:

In [2]:
config = {"configurable": {"user_id": "abc", "thread_id": "abc1"}}

# First let's just say hi to the AI
for event in graph.stream(
    {"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I assist you today?


Given a statement about the user, the LLM can elect to store it for future reference:

In [3]:
for event in graph.stream(
    {"messages": [{"role": "user", "content": "I like pepperoni pizza"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I like pepperoni pizza
================================== Ai Message ==================================
Tool Calls:
  Info (call_imzMyN5FzHd8LeRWVk8Pic5h)
 Call ID: call_imzMyN5FzHd8LeRWVk8Pic5h
  Args:
    fact: likes pepperoni pizza
    topic: Food
================================= Tool Message =================================

Saved!


Note that the application can then reference this statement in a separate thread for the same `user_id` as before:

In [4]:
config = {"configurable": {"user_id": "abc", "thread_id": "abc2"}}

for event in graph.stream(
    {"messages": [{"role": "user", "content": "What should I have for dinner?"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What should I have for dinner?
================================== Ai Message ==================================

How about trying a delicious pepperoni pizza for dinner? It seems like you enjoy pepperoni pizza. Would you like me to remember that for future suggestions?


Inspecting the `MemoryStore`, we can observe the facts that have been stored so far:

In [5]:
user_id = "abc"

kv.data[f"scoped:user_id:info:{user_id}"]

{'0a48f184-af25-4f9e-adb9-f8fb620641e6': {'fact': 'likes pepperoni pizza',
  'topic': 'Food'}}

Note that information is scoped to a `user_id`, so runs only reference facts for the given user. Below, the same query from a different user is given an uninformed answer:

In [6]:
config = {"configurable": {"user_id": "def", "thread_id": "def1"}}

for event in graph.stream(
    {"messages": [{"role": "user", "content": "What should I have for dinner?"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What should I have for dinner?
================================== Ai Message ==================================

What type of cuisine are you in the mood for? Do you have any dietary preferences or restrictions?


## Using an external knowledge-base

If desired, we can update our application to instead reference an external knowledge base. Below, we make these changes:
1. We instantiate a knowledge base, here using a simple vector store (though key-value stores and other knowledge bases will work equally well);
2. We use the built-in `MessagesState` (without the new `info` key);
3. We update the `call_model` node to reference a `user_id` from the run-time configuration and pull from the knowledge base;
4. We update the `update_memory` node to similarly persist to the knowledge base.

We then compile the graph as before, this time without a `MemoryStore`.

In [7]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings


vector_store = InMemoryVectorStore(OpenAIEmbeddings())


def call_model(state: MessagesState, config: RunnableConfig):
    """Call the model."""

    # Fetch user info from knowledge base
    user_id = config["configurable"].get("user_id")
    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = vector_store.similarity_search(
        state["messages"][-1].content, k=3, filter=_filter_function
    )
    # Serialize info
    info = "\n".join(document.page_content for document in documents)
    # Format system prompt
    system_msg = prompt.format(info=info)
    # Call model
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": [response]}


def update_memory(state: MessagesState, config: RunnableConfig):
    """Update the memory."""
    tool_calls = []
    memories = {}
    user_id = config["configurable"].get("user_id")
    # Each tool call is a new memory to save
    for tc in state["messages"][-1].tool_calls:
        # We append ToolMessages (to pass back to the LLM)
        # This is needed because OpenAI requires each tool call be followed by a ToolMessage
        tool_calls.append(
            {"role": "tool", "content": "Saved!", "tool_call_id": tc["id"]}
        )
        # We create a new memory from this tool call
        serialized = str(
            {
                "fact": tc["args"]["fact"],
                "topic": tc["args"]["topic"],
            }
        )
        document = Document(serialized, metadata={"user_id": user_id})
        vector_store.add_documents([document])
        
    # Return the messages and memories to update the state with
    return {"messages": tool_calls}


memory = MemorySaver()

graph = StateGraph(state_schema=MessagesState)
graph.add_node(call_model)
graph.add_node(update_memory)
graph.add_edge("update_memory", "__end__")
graph.add_edge("__start__", "call_model")
graph.add_conditional_edges("call_model", route, ["__end__", "update_memory"])
graph = graph.compile(checkpointer=memory)

Repeating the conversation from before, we see that the application can incorporate information scoped to the user in the same way:

In [8]:
config = {"configurable": {"user_id": "abc", "thread_id": "abc1"}}

for event in graph.stream(
    {"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I assist you today?


In [9]:
for event in graph.stream(
    {"messages": [{"role": "user", "content": "I like pepperoni pizza"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I like pepperoni pizza
================================== Ai Message ==================================
Tool Calls:
  Info (call_FRRoHg5EtNVlKrrD004NYFUn)
 Call ID: call_FRRoHg5EtNVlKrrD004NYFUn
  Args:
    fact: likes pepperoni pizza
    topic: Food
================================= Tool Message =================================

Saved!


In [10]:
config = {"configurable": {"user_id": "abc", "thread_id": "abc2"}}

for event in graph.stream(
    {"messages": [{"role": "user", "content": "What should I have for dinner?"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What should I have for dinner?
================================== Ai Message ==================================

How about trying a delicious pepperoni pizza for dinner? It seems like you enjoy pepperoni pizza!


We can inspect the contents of the knowledge base, this time via the vector store instead of the application state:

In [11]:
entry = list(vector_store.store.values())[0]

{k: v for k, v in entry.items() if k != "vector"}

{'id': 'd29293dd-d9f6-49c9-8898-84b42c1ea02d',
 'text': "{'fact': 'likes pepperoni pizza', 'topic': 'Food'}",
 'metadata': {'user_id': 'abc'}}

Note that as before, runs only reference information for the user specified in the input configuration.

In [12]:
config = {"configurable": {"user_id": "def", "thread_id": "def1"}}

for event in graph.stream(
    {"messages": [{"role": "user", "content": "What should I have for dinner?"}]},
    config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What should I have for dinner?
================================== Ai Message ==================================

I can recommend some dinner options based on your preferences. Do you have any specific dietary restrictions or food preferences?
